MAIN SCRIPT

In [1]:
import numpy as np
from helpers import *
from implementations import *
from clean_data import clean_data
from visualize_features import visualize_features
from cross_validation import *
from poly_reg import *
%load_ext autoreload
%autoreload 2

In [3]:
# Load Data
path_dataset_train = "train.csv"
yb, input_data, ids = load_csv_data(path_dataset_train)

path_dataset_test = "test.csv"
yb_t, input_data_t, ids_t = load_csv_data(path_dataset_test)

In [5]:
# Data Cleaning
tx0, tx1, tx2, tx3, y0, y1, y2, y3, ids0, ids1, ids2, ids3 = clean_data(input_data,yb,ids)

tx0_t, tx1_t, tx2_t, tx3_t, y0_t, y1_t, y2_t, y3_t, ids0_t, ids1_t, ids2_t, ids3_t= clean_data(input_data_t,yb_t,ids_t, test=True) 

In [6]:
# Feature Visualization
features = np.array([1,4,5]) # Features to compare
visualize_features(tx0,y0,features,'Feature_comparision')

In [7]:
# Score the predictions
def find_score(y,y_pred):
    count = 0
    for i in range(len(y_pred)):
        if(y_pred[i] == y[i]):
            count = count + 1       
    return (count/len(y_pred))
    

In [8]:
def construct_predictions(w0, w1, w2, w3, tx0, tx1, tx2, tx3, ids_t,ids0_t, ids1_t,ids2_t,ids3_t):
# Construct the final vector of the predictions using the ids 

    y_pred0 = predict_labels(w0,tx0)
    y_pred1 = predict_labels(w1,tx1)
    y_pred2 = predict_labels(w2,tx2)
    y_pred3 = predict_labels(w3,tx3)

    y_pred = np.ones(len(ids_t))

    for i in range(len(ids0_t)):
        idx = np.where(ids_t==ids0_t[i])[0]
        y_pred[idx] = y_pred0[i]

    for i in range(len(ids1_t)):
        idx = np.where(ids_t==ids1_t[i])[0]
        y_pred[idx] = y_pred1[i]

    for i in range(len(ids2_t)):
        idx = np.where(ids_t==ids2_t[i])[0]
        y_pred[idx] = y_pred2[i]

    for i in range(len(ids3_t)):
        idx = np.where(ids_t==ids3_t[i])[0]
        y_pred[idx] = y_pred3[i]
        
    return y_pred

In [9]:
tx0_train, y0_train, tx0_test, y0_test, ids0_train, ids0_test = split_data(tx0, y0,ids0, 0.7, seed=1)
tx1_train, y1_train, tx1_test, y1_test, ids1_train, ids1_test = split_data(tx1, y1,ids1, 0.7, seed=1)
tx2_train, y2_train, tx2_test, y2_test, ids2_train, ids2_test = split_data(tx2, y2,ids2, 0.7, seed=1)
tx3_train, y3_train, tx3_test, y3_test, ids3_train, ids3_test = split_data(tx3, y3,ids3, 0.7, seed=1)

ridge regression with cross validation

In [ ]:
#find best degrees

degree0 = best_degree(y0_train, tx0_train, 0.000001)
degree1 = best_degree(y1_train, tx1_train, 0.000001)
degree2 = best_degree(y2_train, tx2_train, 0.000001)
degree3 = best_degree(y3_train, tx3_train, 0.000001)

In [ ]:
# use cross validation to find the best lambda for the ridge regression and find the optimal weigths 

lambda0 = best_lambda(y0_train, tx0_train, degree0)
lambda1 = best_lambda(y1_train, tx1_train, degree1)
lambda2 = best_lambda(y2_train, tx2_train, degree2)
lambda3 = best_lambda(y3_train, tx3_train, degree3)



In [ ]:
#build polynomial extensions

tx0_poly_tr = build_poly_f(tx0_train, degree0)
tx1_poly_tr = build_poly_f(tx1_train, degree1)
tx2_poly_tr = build_poly_f(tx2_train, degree2)
tx3_poly_tr = build_poly_f(tx3_train, degree3)

tx0_poly_te = build_poly_f(tx0_test, degree0)
tx1_poly_te = build_poly_f(tx1_test, degree1)
tx2_poly_te = build_poly_f(tx2_test, degree2)
tx3_poly_te = build_poly_f(tx3_test, degree3)

In [ ]:
# find weigths using ridge_regression

_,w0 = ridge_regression(y0_train, tx0_poly_tr, lambda0)
_,w1 = ridge_regression(y1_train, tx1_poly_tr, lambda1)
_,w2 = ridge_regression(y2_train, tx2_poly_tr, lambda2)
_,w3 = ridge_regression(y3_train, tx3_poly_tr, lambda3)

In [ ]:
ids_test =np.concatenate([ids0_test,ids1_test,ids2_test,ids3_test],axis=0)
y_test = np.concatenate([y0_test, y1_test, y2_test, y3_test],axis=0)

In [ ]:
# construct predictions
y_pred = construct_predictions(w0, w1, w2, w3, tx0_poly_te, tx1_poly_te, tx2_poly_te, tx3_poly_te,ids_test,ids0_test, ids1_test,ids2_test,ids3_test)

In [ ]:
find_score(y_pred, y_test)

In [ ]:
create_csv_submission(ids_t, y_pred, 'submission.csv')